
# <p style="text-align: center;">EE 380L: Data Mining</p>
# <p style="text-align: center;">Assignment 5</p>
## <p style="text-align: center;">Total points: 40</p>
## <p style="text-align: center;">Due: Thursday, April 24th, submitted via Canvas by 11:59 pm</p>

Your homework should be written in a **Jupyter notebook**. You may work in groups of two if you wish. Only one student per team needs to submit the assignment on Canvas.  But be sure to include name and UTID for both students.  Homework groups will be created and managed through Canvas, so please do not arbitrarily change your homework group.  

Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting. (%matplotlib inline)

## Name: Xiaocheng Shen
## UT EID: xs2948

## Name: Dawei Liang
## UT EID: dl33629

# Problem 1 - Random Forest vs Boosting - Regression (15pts)

In this question, we will compare performance of different ensemble methods for regression problems: [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html), [Gradient Boosting Regressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) (GBR), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor). We will use the Appliances energy prediction dataset for this problem https://archive.ics.uci.edu/ml/datasets/Appliances+energy+prediction. Use random_state = 42 for the algorithms.

1. Use a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html). Find the best parameters (*n_estimators*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report the accuracy of your model in terms of RMSE. (5pts)

2. Use [Gradient Boosting Regressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) (GBR). Find the best parameters (including *n_estimators,* *max_depth* and* learning_rate*), and report corresponding RMSE for each algorithm. (5pts)

3. Use [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor) for predicting the targets. Find the best parameters (*n_estimators*, *learning_rate*). Report the accuracy of your model in terms of RMSE. (5pts)

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import (RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor)

In [10]:
data = pd.read_csv('energydata_complete.csv') 

y = data['Appliances']
X = data.drop(['date','Appliances', 'rv1', 'rv2'], axis = 1)


from dateutil import parser
timeData = np.array(data['date'])

days = []
hours = []
for line in range(len(timeData)):
    day = parser.parse(timeData[line]).weekday()
    hour = parser.parse(timeData[line]).hour
    days.append(day)
    hours.append(hour)
    
X = pd.concat([X, pd.get_dummies(days), pd.get_dummies(hours)], axis = 1)


x_train, x_test, y_train, y_test = train_test_split(X,y,test_size = 0.33, random_state=42)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(13222, 56) (13222,) (6513, 56) (6513,)


In [11]:
parameters = {'n_estimators':[1,5,10,20,50]}
rf = RandomForestRegressor(random_state=42,)
rg = GridSearchCV(rf, parameters)
model = rg.fit(x_train, y_train)
rmse = np.sqrt(mean_squared_error(y_test, model.predict(x_test)))
print('RF:',model.best_params_ )
print('RF:rmse,',rmse)


RF: {'n_estimators': 50}
RF:rmse, 71.63264911664761


In [13]:
learning_rate = [0.01,0.05,0.1,0.2,0.5]
max_depth = [1,2,3,4,5]
rmse_list = [0] * 25

for i in range(5):
    for j in range(5):
        rg2 = GradientBoostingRegressor(learning_rate=learning_rate[j], max_depth=max_depth[i])
        rg2.fit(x_train, y_train)
        rmse_list[i * 5 + j] = np.sqrt(mean_squared_error(y_test, rg2.predict(x_test)))

best_i = rmse_list.index(min(rmse_list))
learning_rate, max_depth = learning_rate[best_i%5], max_depth[int(best_i/5)]

parameters = {'n_estimators':[10,50,100,200,300]}
rf2 = GradientBoostingRegressor(learning_rate=learning_rate, max_depth=max_depth)
rg2 = GridSearchCV(rf2, parameters)
model2 = rg2.fit(x_train, y_train)
rmse = np.sqrt(mean_squared_error(y_test, model2.predict(x_test)))
print('GBDT:', model2.best_params_, 'best learning rate:',learning_rate, 'best depth:',max_depth)
print('GBDT:rmse',rmse)

GBDT: {'n_estimators': 200} best learning rate: 0.5 best depth: 5
GBDT:rmse 75.8764397161184


In [14]:
learning_rate = [0.1,0.5,1,2,5]
rmse_list = [0] * 5

for i in range(5):
    rg3 = AdaBoostRegressor(learning_rate=learning_rate[i])
    rg3.fit(x_train, y_train)
    score_list[i] = np.sqrt(mean_squared_error(y_test, rg3.predict(x_test)))

best_i = rmse_list.index(min(rmse_list))
learning_rate= learning_rate[best_i%5]

parameters = {'n_estimators':[1,10,50,100,200]}
rf3 = AdaBoostRegressor(learning_rate=learning_rate)
rg3 = GridSearchCV(rf3, parameters)
model3 = rg3.fit(x_train, y_train)
rmse = np.sqrt(mean_squared_error(y_test, model3.predict(x_test)))
print('Adaboost:',model3.best_params_,'best learning rate:',learning_rate)
print('Adaboost:rmse,',rmse)

Adaboost: {'n_estimators': 10} best learning rate: 0.1
Adaboost:rmse, 96.81017955674213


# Solution

# Problem 2 - Random Forest vs Boosting - Classification (15 pts)
In this question, we will compare performance of different ensemble methods for classification problems: [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), [Gradient Boosting Decision Tree](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (GBDT), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier). As in HW4, you will be classifying signal vs background in the MAGIC Gamma Telescope Data Set. The data has been split into training and test as well as standardized for you.

1. Use a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) for the classification. Set the random_state to 42. Find the best parameters (by varying *n_estimators* and *criterion*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report your testing accuracy ([accuracy_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)) and [roc_auc_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score). You will need [predict_proba](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.predict_proba) for roc_auc_score. (4pts)

2. Use [Gradient Boosting Decision Tree](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (GBDT), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) for the spam classification problem. Again, find the best parameters (including *n_estimators, learning_rate,* and *max_depth (GBDT only)*), and report corresponding accuracy_score and roc_auc_score on the test data for each algorithm. (8pts)

3. Point out one advantage and one disadvantage of Random Forest compared to GBDT (3pts)

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import (accuracy_score,roc_auc_score)
from sklearn.ensemble import (RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier)
from sklearn import preprocessing

In [3]:
# load data
# train
train = pd.read_csv('magic_train.csv', header=None)
y_train = train.values[:,10]
y_train[y_train == 'g'] = 0
y_train[y_train == 'h'] = 1
y_train = y_train.astype(float)
x_train = train.values[:,:10]

# test
test = pd.read_csv('magic_test.csv', header=None)
y_test = test.values[:,10]
y_test[y_test == 'g'] = 0
y_test[y_test == 'h'] = 1
y_test = y_test.astype(float)
x_test = test.values[:,:10]

# standardize the data
x_train = preprocessing.scale(x_train)
x_test = preprocessing.scale(x_test)

/Users/xiaochengshen/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [4]:
parameters = {'n_estimators':[1,10,50,100,200], 'criterion':['gini','entropy']}
rf = RandomForestClassifier(random_state=42,)
clf = GridSearchCV(rf, parameters)
model = clf.fit(x_train, y_train)
print('RF:',model.best_params_ )
print('RF:accuracy_score,',accuracy_score(y_test, model.predict(x_test)))
print('RF:roc_auc_score,',roc_auc_score(y_test, model.predict_proba(x_test)[:,1]))


RF: {'criterion': 'gini', 'n_estimators': 200}
RF:accuracy_score, 0.8768318213538032
RF:roc_auc_score, 0.9321763568768894


In [6]:
learning_rate = [0.01,0.05,0.1,0.2,0.5]
max_depth = [1,2,3,4,5]
score_list = [0] * 25

for i in range(5):
    for j in range(5):
        clf2 = GradientBoostingClassifier(learning_rate=learning_rate[j], max_depth=max_depth[i])
        clf2.fit(x_train, y_train)
        score_list[i * 5 + j] = clf.score(x_test, y_test)

best_i = score_list.index(max(score_list))
learning_rate, max_depth = learning_rate[best_i%5], max_depth[int(best_i/5)]

parameters = {'n_estimators':[1,10,50,100,200]}
rf2 = GradientBoostingClassifier(learning_rate=learning_rate, max_depth=max_depth)
clf2 = GridSearchCV(rf2, parameters)
model2 = clf2.fit(x_train, y_train)
print('GBDT:', model2.best_params_, 'best learning rate:',learning_rate, 'best depth:',max_depth)
print('GBDT:accuracy_score',accuracy_score(y_test, model2.predict(x_test)))
print('GBDT:roc_auc_score',roc_auc_score(y_test, model2.predict_proba(x_test)[:,1]))



learning_rate = [0.1,0.5,1,2,5]
score_list = [0] * 5

for i in range(5):
    clf3 = GradientBoostingClassifier(learning_rate=learning_rate[i])
    clf3.fit(x_train, y_train)
    score_list[i] = clf.score(x_test, y_test)

best_i = score_list.index(max(score_list))
learning_rate= learning_rate[best_i%5]

parameters = {'n_estimators':[1,10,50,100,200]}
rf3 = AdaBoostClassifier()
clf3 = GridSearchCV(rf3, parameters)
model3 = clf3.fit(x_train, y_train)
print('Adaboost:',model3.best_params_,'best learning rate:',learning_rate)
print('Adaboost:accuracy_score,',accuracy_score(y_test, model3.predict(x_test)))
print('Adaboost:roc_auc_score',roc_auc_score(y_test, model3.predict_proba(x_test)[:,1]))

GBDT: {'n_estimators': 200} best learning rate: 0.01 best depth: 1
GBDT:accuracy_score 0.7418004187020237
GBDT:roc_auc_score 0.8381682548439772
Adaboost: {'n_estimators': 200} best learning rate: 0.1
Adaboost:accuracy_score, 0.8473482205163991
Adaboost:roc_auc_score 0.9000435473322483


GBDT is based on weak learners which are shallow trees, sometimes the trees are as small as with two leaves. GBDT reduces error mainly by reducing bias.


Random Forest uses fully grown decision trees (low bias, high variance). It reduces error by reducing variance. The trees are made uncorrelated to maximize the decrease in variance without reducing bias. 

## Solution

# Question 3: Visualization using Bokeh (10 pts)

In this problem, you'll build an interactive visualization. Bokeh is a Python interactive visualization library that targets modern web browsers for presentation. For more information on Bokeh, see http://bokeh.pydata.org/en/latest/. The problem statement is as follows:

Using the "nbasalaries_new.csv" data set, your goal is to build a Bokeh visualization which allows the user to explore how salary (on a log scale) varies with points per game (PSG) and age. You will create a visualization that allows the user to toggle the X axis of a scatter plot between PSG and age, with the y-axis always being log Salary. Also add the hover tool so that if the user hovers over a datapoint in the plot a window pops up that shows the player name, team, position, salary, and the current x variable (PSG or age) depending on the current tab.  Add the ability to Zoom in/out.

Hints: 
1. see: http://bokeh.pydata.org/en/latest/docs/user_guide/tools.html#basic-tooltips for hover and zoom tool examples.
2. See: http://bokeh.pydata.org/en/latest/docs/reference/plotting.html. Look for the scatter API.
3. See: http://bokeh.pydata.org/en/0.10.0/docs/user_guide/styling.html#labels. For labeling axes.
4. See: https://bokeh.pydata.org/en/latest/docs/user_guide/categorical.html  for how to use jitter transform
5. Use output_file() from Bokeh to output the plot to an external file. Submit this file as well.

In [43]:
import pandas as pd
import numpy as np
from bokeh.models.widgets import Panel, Tabs
from bokeh.models import CustomJS, ColumnDataSource, HoverTool, BoxZoomTool, Jitter
from bokeh.plotting import figure, output_notebook, show, output_file
from bokeh.io import output_file, show
from bokeh.models.widgets import Toggle

data = pd.read_csv("nbasalaries_new.csv")
data["logsalary"] = data.SALARY.apply(np.log)

## Solution